# Obtenção dos dados do scopus através do uso de uma API


## Caminhos dos dados 

In [1]:
#Colocar os path correto

path = r"C:\Users\Catarina\Documents\GitHub\ICD"

path_dados = r"C:\Users\Catarina\Documents\GitHub\ICD"

## Módulos de interesse

In [2]:
import requests
import json

import pickle

import math 
import pandas as pd

## Funções auxiliares

In [3]:
# A função "jprint()" tem como objetivo "imprimir" o json (formato de resposta escolhido / solicitado à API) 
# O formato ".json" é uma linguagem utilizada para representação de dados num formato do tipo "chave-valor" - numa estrutura de dados similar a um "dicionário", 
# disponibilizada na linguagem Python. 
# Os dados, tal como numa base de dados usual, podem ter uma estrutura relacional; é usual que estas relações possam ser representadas num formato
# hierárquico. O formato json é assim compatível com esta representação relacional hierárquica, sendo a navegação entre os diferentes níveis definida pelas chaves, 
# que correspondem aos nós da representação hierárquica em rede dos referidos dados codificados no formato json
# Esta função permite representar a estrutura hierárquica do json, através da "impressão" em formato textual, utilizando tabulações por forma a alinhar as chaves que se encontram
# em cada nível hierárquico

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=6)
    print(text)



## Configuração Chave API

In [4]:
pip install python-decouple

  Using cached python_decouple-3.8-py3-none-any.whl (9.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from decouple import Config, RepositoryEnv

DOTENV_FILE = r"C:\Users\Catarina\Documents\GitHub\ICD\.env"
env_config = Config(RepositoryEnv(DOTENV_FILE))

# use the ConfigEnv().get() method as you normally would since 
# decouple.config uses that internally. 
# i.e. config('SECRET_KEY') = env_config.get('SECRET_KEY')
MY_SCOPUS_API_KEY = env_config.get('066bdb7b3de4992deb102287c3232c46')

print('my scopus api key is :', MY_SCOPUS_API_KEY) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Catarina\\Documents\\GitHub\\ICD\\.env'

# RECOLHA DE DADOS - Parte 1
Pesquisa e recolha de resultados com a API Scoupus API

## Análise exploratória dos resultados de pesquisa

### Pesquisa exploratória API

In [ ]:
# IMPORTANTE: https://dev.elsevier.com/sc_search_tips.html

user_query = "data mining housing automated valuation model"
cursor = "*"

results = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_SCOPUS_API_KEY},
                         
                    params={    'query' : user_query,
                                #'count' : 1000,
                                #'start' : "25",
                                #'cursor' : '*',
                        }

                        )

print(results.url)



NameError: name 'MY_SCOPUS_API_KEY' is not defined

### Identificar número total de resultados obtidos na pesquisa

In [ ]:
number_of_articles_retrieved = results["search-results"]["opensearch:totalResults"]
number_of_articles_retrieved

: 

In [ ]:
number_of_articles_perResultPage = results["search-results"]["opensearch:itemsPerPage"]
number_of_articles_perResultPage

: 

## Implementação global - Parte 1 (DOIs)
Recolha de resultados individuais da API, seleção de dados de interesse, criação de estrutura de dados amigável

In [ ]:
user_query = "data mining housing automated valuation model"
LEVEL0_FIELDS_OF_INTEREST = ['dc:title', "prism:doi", 'dc:creator']
AFFILIATION_FIELDS_OF_INTEREST = ['affilname', 'affiliation-city', 'affiliation-country']

df_results_list = pd.DataFrame(columns=LEVEL0_FIELDS_OF_INTEREST+AFFILIATION_FIELDS_OF_INTEREST)

start = 0

while start <= int(number_of_articles_retrieved):
    
    
    response = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY},
                         
                    params={    'query' : user_query,
                                'start' : start
                                #'cursor' : cursor,
                        }

                        )
    print(i)
    print(response.url)
    print(response.status_code)



    if(response.status_code == 200):

        results_aux = response.json()

        df_level1_aux = pd.DataFrame.from_records(results_aux["search-results"]['entry'] )
        df_level1_aux = df_level1_aux[LEVEL0_FIELDS_OF_INTEREST]

        if 'affiliation' in results["search-results"]['entry'][0]:
            
            df_affil_aux = pd.json_normalize(results["search-results"],['entry', 'affiliation'] )
            df_affil_aux = df_affil_aux[AFFILIATION_FIELDS_OF_INTEREST]
            df_all_aux = pd.merge(df_level1_aux, df_affil_aux, how='left', left_index=True, right_index=True)
            df_results_list = df_results_list.append(df_all_aux, ignore_index=True)

        else:

            df_affil_aux = pd.DataFrame(np.nan, index=[0],columns=AFF_COL_NAMES)
            df_all_aux = pd.merge(df_level1_aux, df_affil_aux, how='left', left_index=True, right_index=True)
            df_results_list = df_results_list.append(df_all_aux, ignore_index=True)
            
    start = start+int(number_of_articles_perResultPage)

    
    

: 

In [ ]:
df_results_list[:300]

: 

In [ ]:
df_results_list["prism:doi"]

: 

# RECOLHA DE DADOS - Parte 2


## Recolha de resumos (dos artigos) 

Recolha do contéudo dos resumos dos artigos selecionados com a API Scopus (usando o identificador DOI guardado anteriormente)

NOTA: Como facilmente se verifica na [página](https://dev.elsevier.com/api_docs.html) os resumos podem ser obtidos de duas formas:
* Através do serviço «ScienceDirect APIs» e especificamente da API «Article Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/ArticleRetrievalAPI.wadl) ) descrita anteriormente
* Através do serviço «Scopus APIs» (usado anteriormente para fazer a pesquisa) e especificamente da API «Abstract Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/AbstractRetrievalAPI.wadl))

## Implementação Global - Parte 2

In [ ]:
dois = df_results_list['prism:doi']
dois

: 

In [ ]:
abstract_list = []

i=0
for x in dois :
    print(x)
    response_abst = requests.get("https://api.elsevier.com/content/abstract/doi/"+str(x),
                    headers={
                    'Accept': 'application/json',
                    'X-ELS-APIKey': MY_API_KEY},
                    
                    params={   }
                )
    
    print(response_abst.url)
    print(response_abst.status_code)
    print(i)
    i=i+1

    if(response_abst.status_code == 200):
        results_abst = response_abst.json()
        if 'dc:description' in results_abst["abstracts-retrieval-response"][ "coredata"]:
            abstract_list.append( results_abst["abstracts-retrieval-response"][ "coredata"]['dc:description'] )
        else:
            abstract_list.append("NULL")
    else:
        abstract_list.append("NULL")




: 

In [ ]:
abstract_list[0]

: 

In [ ]:
type(abstract_list[0])

: 

# GUARDAR DADOS RECOLHIDOS E PROCESSADOS

In [ ]:
df_results_list.to_csv(path_dados+'search_results.csv',  index=False, encoding = "utf-8")

pd.DataFrame(abstract_list,  columns =['Abstract']).to_csv(path_dados+'search_results_abstracts.csv', index=False, encoding = "utf-8")

: 

In [ ]:
# Save the file
#pickle.dump( df_results_list, file = open(path_dados+"ICD2122_TextMining_DataStructures.pickle", "wb"))

# Reload the file
#test_grouped_df_reloaded = pickle.load(open(path_dados+"ICD2122_TextMining_DataStructures.pickle", "rb"))

: 